Пакет SURPRISE:

используйте данные MovieLens 1M,
можно использовать любые модели из пакета,
получите RMSE на тестовом сете 0,87 и ниже.


In [ ]:
pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163014 sha256=9fb34eb3dff23d10672b0f6caadf66136f887bd6a9164b0662bb0bcf968e7262
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import pandas as pd

In [ ]:
movies =  pd.read_csv('/content/drive/MyDrive/rek_syst/ml-latest-small/movies.csv')
rating = pd.read_csv('/content/drive/MyDrive/rek_syst/ml-latest-small/ratings.csv')

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
rating.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
mov_rat = movies.merge(rating, on='movieId').reset_index(drop=True)

In [ ]:
mov_rat.dropna()
mov_rat.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


 Далее использую датасет 100К загруженный с сайта "https://grouplens.org/datasets/movielens/" в разделе older dataset.

 Так как он уже был подготовлен, не стал на нем проводить подготовительную часть.

 В нем используются не названия, а ID фильмов, но подтянуть фильм зная ID не так сложно

In [ ]:
columns =  ['userId','itemId','rating','timestamp']
datau = pd.read_csv('/content/drive/MyDrive/dataset/ml-100k/u.data',sep='\t',names = columns)

Загрузка фильмов, на случай если понадобится подтянуть название.

In [ ]:
columnsf = ['movie id' , 'movie title' , 'release date' , 'video release date' ,
              'IMDb URL' , 'unknown' , 'Action' , 'Adventure' , 'Animation' ,
              'Childrens' , 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western']
datafilm = pd.read_csv('/content/drive/MyDrive/dataset/ml-100k/u.item',sep='|',names = columnsf,encoding='latin-1')

In [ ]:
datafilm.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
datau.head()

,userId,itemId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
data = pd.DataFrame({
    'uid' : mov_rat.userId,
    'iid' : mov_rat.movieId,
    'rating' : mov_rat .rating
})

In [ ]:
data.head()

,uid,iid,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [ ]:
min1 = data.rating.min()
min1

0.5

In [ ]:
max1 = data.rating.max()
max1

5.0

In [ ]:
from surprise import KNNBasic, KNNWithMeans, KNNBaseline, KNNWithZScore, NMF, SlopeOne, CoClustering
from surprise import Dataset, accuracy, Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold

In [ ]:
reader = Reader(rating_scale=(min1 , max1))

In [ ]:
datasur = Dataset.load_from_df(data,reader)

###KNNWithMeans

In [ ]:
def dz (algoritm,nm):
  name_alg = ['cosine', 'msd','pearson','pearson_baseline']
  u_b = [True,False]
  kf = KFold(n_splits=5,random_state = 13)
  algor = pd.DataFrame({'number_alg':[],'method':[],'user_base':[],'RMSE':[]})
  algoritmm = str(algoritm)
  i=1
  for name_algi in name_alg:
    for u_bi in u_b:
      for trainset, testset in kf.split(datasur):
        alg = algoritm(k = 50, min_k = 45, sim_options={
          'name':name_algi,
          'user_based':u_bi
           },verbose = False)
        alg.fit(trainset)
        test_pred = alg.test(testset,verbose = False)
        rmsex = accuracy.rmse(test_pred, verbose = False)
        i = str(i)
        algor.loc[len(algor.index)]=[i,name_algi,u_bi,rmsex]
        i = int(i)
        globals()[f'{nm}_{i}'] = alg
        i=i+1
        del test_pred
        del rmsex
  del i
  return algor

In [ ]:
def dz1 (algoritm,nm):
  name_alg = ['cosine', 'msd','pearson','pearson_baseline']
  u_b = [True,False]
  kf = KFold(n_splits=5,random_state = 13)
  algor = pd.DataFrame({'number_alg':[],'method':[],'user_base':[],'RMSE':[]})
  algoritmm = str(algoritm)
  i=1
  for name_algi in name_alg:
    for u_bi in u_b:
      for trainset, testset in kf.split(datasur):
        alg = algoritm()
        alg.fit(trainset)
        test_pred = alg.test(testset,verbose = False)
        rmsex = accuracy.rmse(test_pred, verbose = False)
        i = str(i)
        algor.loc[len(algor.index)]=[i,name_algi,u_bi,rmsex]
        i = int(i)
        globals()[f'{nm}_{i}'] = alg
        i=i+1
        del test_pred
        del rmsex
  del i
  return algor

In [ ]:
nm = 'KNNWithMeans'
algoritm_KNNWithMeans = dz(KNNWithMeans,nm)

In [ ]:
algoritm_KNNWithMeans.sort_values('RMSE').head()

,number_alg,method,user_base,RMSE
15,16,msd,False,0.943976
16,17,msd,False,0.945671
35,36,pearson_baseline,False,0.946243
17,18,msd,False,0.947493
19,20,msd,False,0.947652


### KNNBasic

In [ ]:
nm = 'KNNBasic'
algoritm_KNNBasic = dz(KNNBasic,nm)

In [ ]:
algoritm_KNNBasic.sort_values('RMSE').head()

,number_alg,method,user_base,RMSE
15,16,msd,False,0.990025
17,18,msd,False,0.990444
16,17,msd,False,0.990884
19,20,msd,False,0.991365
18,19,msd,False,0.995290


###KNNBaseline

In [ ]:
nm = 'KNNBaseline'
algoritm_KNNBaseline = dz(KNNBaseline,nm)

In [ ]:
algoritm_KNNBaseline.sort_values('RMSE').head()

,number_alg,method,user_base,RMSE
35,36,pearson_baseline,False,0.919296
39,40,pearson_baseline,False,0.919381
37,38,pearson_baseline,False,0.919721
36,37,pearson_baseline,False,0.920734
30,31,pearson_baseline,True,0.922751


### KNNWithZScore

In [ ]:
nm = 'KNNWithZScore'
algoritm_KNNWithZScore = dz(KNNWithZScore,nm)

In [ ]:
algoritm_KNNWithZScore.sort_values('RMSE').head()

,number_alg,method,user_base,RMSE
15,16,msd,False,0.945614
16,17,msd,False,0.947416
35,36,pearson_baseline,False,0.948221
37,38,pearson_baseline,False,0.949167
17,18,msd,False,0.949274


### NMF

In [ ]:
nm = 'NMF'
algoritm_NMF = dz1(NMF,nm)

In [ ]:
algoritm_NMF.sort_values('RMSE').head()

,number_alg,method,user_base,RMSE
2,3,cosine,True,0.957697
35,36,pearson_baseline,False,0.958353
1,2,cosine,True,0.958573
30,31,pearson_baseline,True,0.958990
15,16,msd,False,0.959113


### SlopeOne

In [ ]:
nm = 'SlopeOne'
algoritm_SlopeOne = dz1(SlopeOne,nm)

In [ ]:
algoritm_SlopeOne.sort_values('RMSE').head()

,number_alg,method,user_base,RMSE
0,1,cosine,True,0.941197
35,36,pearson_baseline,False,0.941197
30,31,pearson_baseline,True,0.941197
5,6,cosine,False,0.941197
25,26,pearson,False,0.941197


### Co-clustering

In [ ]:
nm = 'Co-clustering'
algoritm_Co_clustering = dz1(CoClustering,nm)

In [ ]:
algoritm_Co_clustering.sort_values('RMSE').head()

,number_alg,method,user_base,RMSE
19,20,msd,False,0.953549
25,26,pearson,False,0.953958
14,15,msd,True,0.954019
30,31,pearson_baseline,True,0.954243
2,3,cosine,True,0.954954


Попробовав все варианты остановился на KNNBaseline с параметрами:

name: pearson_baseline и user_based: False

In [ ]:
def dz3 (algoritm,nm):
  name_alg = ['pearson_baseline']
  u_b = [False]
  kf = KFold(n_splits=5,random_state = 13)
  algor = pd.DataFrame({'number_alg':[],'method':[],'user_base':[],'RMSE':[]})
  algoritmm = str(algoritm)
  i=1
  for name_algi in name_alg:
    for u_bi in u_b:
      for trainset, testset in kf.split(datasur):
        alg = algoritm(k = 50, min_k = 45, sim_options={
          'name':name_algi,
          'user_based':u_bi},
                       bsl_options={
          'method':'als',
          'reg_i':5,
          'reg_u':25,
          'n_epochs': 10
           },verbose = False)
        alg.fit(trainset)
        test_pred = alg.test(testset,verbose = False)
        rmsex = accuracy.rmse(test_pred, verbose = False)
        i = str(i)
        algor.loc[len(algor.index)]=[i,name_algi,u_bi,rmsex]
        i = int(i)
        globals()[f'{nm}_{i}'] = alg
        i=i+1
        #del test_pred
        #del rmsex
  del i
  return algor

In [ ]:
nm = 'KNNBaseline'
algoritm_KNNBaseline = dz3(KNNBaseline,nm)
algoritm_KNNBaseline.sort_values('RMSE').head()

,number_alg,method,user_base,RMSE
0,1,pearson_baseline,False,0.918175
2,3,pearson_baseline,False,0.919453
4,5,pearson_baseline,False,0.919454
1,2,pearson_baseline,False,0.920306
3,4,pearson_baseline,False,0.926937


### SVDpp CROSS_VALIDATION

In [ ]:
#Данный блок просто для удобства при перезапуске
!pip install surprise
import pandas as pd
movies =  pd.read_csv('/content/drive/MyDrive/rek_syst/ml-latest-small/movies.csv')
rating = pd.read_csv('/content/drive/MyDrive/rek_syst/ml-latest-small/ratings.csv')
mov_rat = movies.merge(rating, on='movieId').reset_index(drop=True)
mov_rat.dropna()
data = pd.DataFrame({
    'uid' : mov_rat.userId,
    'iid' : mov_rat.movieId,
    'rating' : mov_rat.rating
})
min1 = data.rating.min()
max1 = data.rating.max()
from surprise import Dataset, accuracy, Reader
from surprise.model_selection import train_test_split
from surprise import Dataset, SVDpp
from surprise.model_selection import cross_validate
reader = Reader(rating_scale=(min1 , max1))
datasur = Dataset.load_from_df(data,reader)

### попытка перебора параметров в цикле

In [ ]:
def frange1(start, stop, step):
	    i = start
	    while i < stop:
	        yield i
	        i += step
def frange2(start, stop, step):
	    i = start
	    while i < stop:
	        yield i
	        i += step
def frange3(start, stop, step):
	    i = start
	    while i < stop:
	        yield i
	        i += step
def frange4(start, stop, step):
	    i = start
	    while i < stop:
	        yield i
	        i += step

In [ ]:
i=1
par = []
for im in frange1(0,2,0.5):
  for isd in frange2(0.1,0.5,0.1):
    for la in frange3(0.003,0.009,0.001):
      for ra in frange4(0.02,0.05,0.01):
        algoritm_cv=SVDpp(n_factors=20,
                  n_epochs=20,
                  init_mean=im,
                  init_std_dev=isd,
                  lr_all=la,
                  reg_all=ra,
                  random_state=13,
                  verbose=False,
                  cache_ratings=False)
        locals()[f'resall_{i}'] = pd.DataFrame(cross_validate(algoritm_cv, datasur, measures=["RMSE", "MAE"], cv=5, verbose=False))
        locals()[f'parparall_{i}']= []
        locals()[f'parparall_{i}'].extend([im,'|',isd,'|',la,'|',ra,'|'])
        i = i+1


In [ ]:
nn=[]
for w in range(1,i,1):
  mmm = locals()[f'resall_{w}'].test_rmse.min()
  nn.append(mmm)
  print(w)
  print(mmm)
  print('-----------------')
min(nn)

### ручной подбор

In [ ]:
algoritm_cv=SVDpp(n_factors=20,
                  n_epochs=20,
                  init_mean=0,
                  init_std_dev=0.1,
                  lr_all=0.007,
                  reg_all=0.02,
                  random_state=13,
                  verbose=False,
                  cache_ratings=False)

In [ ]:
res = pd.DataFrame(cross_validate(algoritm_cv, datasur, measures=["RMSE", "MAE"], cv=5, verbose=False))
res

,test_rmse,test_mae,fit_time,test_time
0,0.857101,0.657250,118.772113,13.342797
1,0.864617,0.665020,110.728927,13.512837
2,0.862324,0.660181,111.865772,13.217732
3,0.864743,0.661370,112.368915,13.137630
4,0.863352,0.662737,111.969535,12.736243


In [ ]:
res.test_rmse.min()

0.8571010934155424